In [73]:
from requests import get
url = 'http://www.angelfire.com/planet/hawarya1/spices-ethiopia.htm'
response = get(url)
print(response.text[:200])

<html>

<head>
<meta name="GENERATOR" content="Microsoft FrontPage 5.0">
<meta name="ProgId" content="FrontPage.Editor.Document">
<meta http-equiv="Content-Type" content="text/html; charset=windo


In [2]:
from bs4 import BeautifulSoup
html_soup = BeautifulSoup(response.text, 'html.parser')
type(html_soup)

bs4.BeautifulSoup

In [33]:
spice_containers = html_soup.find_all('p', class_ = 'MsoNormal')
print(type(spice_containers))
print(len(spice_containers))

<class 'bs4.element.ResultSet'>
120


In [161]:
#let's look at the first ingredient-starts at 17
first_spice = spice_containers[17]
first_spice

<p class="MsoNormal"><span style="font-size: 10.0pt; font-family: Arial Narrow">\r\nN:\xa0 <b>Abyssinian thyme/<u>Tosign</u></b>\xa0 </span></p>

In [164]:
#access the amharic name within <b> tag
print first_spice.b.text
#access the english name within <u> tag under the <b> tag
print first_spice.b.u.text

[u'Abyssinian thyme', u'Tosign']
Tosign


In [52]:
#let's look at the active ingredient list for the ingredient
ingredient = spice_containers[18]
ingredient

<p class="MsoNormal" style="text-indent: -.25in; margin-left: .25in">\n<span style="font-size: 10.0pt; font-family: Arial Narrow">C:\xa0 Volatile \r\n(essential) oils including cymene, methylchevicol, borneol, cineole, terpinene, \r\nthymol, linalool, carvacol; flavonoids; triterpenes; tannins</span></p>

In [71]:
#access the description of ingredients inside the <span>
ingredient.span.text

u'C:\xa0 Volatile \r\n(essential) oils including cymene, methylchevicol, borneol, cineole, terpinene, \r\nthymol, linalool, carvacol; flavonoids; triterpenes; tannins'

In [84]:
#strip string of non breaking space and newline 
ingredient.span.text.replace('\r\n','').replace(u'C:\xa0 ',u'').strip()

u'Volatile (essential) oils including cymene, methylchevicol, borneol, cineole, terpinene, thymol, linalool, carvacol; flavonoids; triterpenes; tannins'

In [75]:
#let's look at the health benifts for the ingredient
benefit = spice_containers[19]
benefit

<p class="MsoNormal" style="text-indent: -.25in; margin-left: .25in">\n<span style="font-size: 10.0pt; font-family: Arial Narrow">B:\xa0 For respiratory \r\nproblems (cough, bronchitis, soar throat), gastrointestinal disorders (colic, \r\ndyspepsia, gastritis, flatulence, diarrhea); halitosis; rheumatism; as appetite \r\nstimulant, diuretic, anthelminthic, [externally counterirritant, antimicrobial, \r\nanti-inflammatory certain skin disorders].\xa0 </span></p>

In [77]:
#access the description of benefits inside the <span>
benefit.span.text

u'B:\xa0 For respiratory \r\nproblems (cough, bronchitis, soar throat), gastrointestinal disorders (colic, \r\ndyspepsia, gastritis, flatulence, diarrhea); halitosis; rheumatism; as appetite \r\nstimulant, diuretic, anthelminthic, [externally counterirritant, antimicrobial, \r\nanti-inflammatory certain skin disorders].\xa0 '

In [85]:
#strip string of non breaking space and newline 
benefit.span.text.replace('\r\n','').replace(u'B:\xa0 ',u'').strip()

u'For respiratory problems (cough, bronchitis, soar throat), gastrointestinal disorders (colic, dyspepsia, gastritis, flatulence, diarrhea); halitosis; rheumatism; as appetite stimulant, diuretic, anthelminthic, [externally counterirritant, antimicrobial, anti-inflammatory certain skin disorders].'

In [167]:
# create a function that creates 4 lists of spices with ingredients and benefits 
def spice_description(containers):
    name=containers[17::4]
    eng_name=[]
    amh_name=[]    
    for tag in name:
        common_name=tag.b.text.strip()
        eng_name.append(common_name.split('/')[0])
        if not 'Not well defined' in common_name:
            amh_name.append(tag.b.u.text.strip())
        else:      
            amh_name.append(None)       

    desc=containers[18::4]
    ingr=[]
    for i in desc:
        ingr.append(i.span.text.replace('\r\n','').replace(u'C:\xa0 ',u'').strip())

    hbenf=containers[19::4]  
    benf=[]
    for j in hbenf:
        benf.append(j.span.text.replace('\r\n','').replace(u'B:\xa0 ',u'').strip())


    return eng_name, amh_name, ingr, benf

english_name, amharic_name, ingredients_list, benefits = spice_description(spice_containers)
#english_name    
len(english_name),len(amharic_name),len(ingredients), len(benefits)


(26, 26, 26, 26)

In [171]:
#create a df from lists
import pandas as pd
spice_benefits=pd.DataFrame({'spice': amharic_name,
                            'english_name': english_name,
                            'ingredients': ingredients_list,
                            'benefits':benefits
                            }, columns=['spice','english_name','ingredients','benefits'])
spice_benefits.head()


,spice,english_name,ingredients,benefits
0,Tosign,Abyssinian thyme,"Volatile (essential) oils including cymene, me...","For respiratory problems (cough, bronchitis, s..."
1,Insilal,Anise,"Volatile oils such as anethole, methylchavicol...","Carminative relieving flatulence, indigestion ..."
2,Besobila,Basil,"Volatile oils of various types: estragole, met...",Gastrointestinal problems; for colds; for peri...
3,Nech azmud,Bishops Weed,"Furanocumarins, flavonoids, tannins","Carminative, generally calming the digestive s..."
4,Tikur azmud,Black cumin or black seed,"Volatile oils of various types, saponin; thymo...","Carminative relieving flatulence, stomach pain..."
